In [24]:
import os
import json
import time
import requests

data_dir = 'data'
with open(os.path.join(data_dir, 'sites', "PlanetScope_API_key.txt"), "r") as file:
   PLANET_API_KEY = file.read()  # Read entire file content

# testing new Planetscope API key

In [28]:
# data API testing
URL = 'https://api.planet.com/data/v1'

# session
session = requests.Session()

session.auth = (PLANET_API_KEY, "")

res = session.get(URL)
res.status_code

def p(data):
    print(json.dumps(data, indent = 2))

In [100]:
data_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte" : "2017-11-30T00:00:00.000Z",
        'lte' : '2018-01-30T00:00:00.000Z'
    }
}

In [57]:
geometry = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": {
        "type": 'Polygon',
        "coordinates" :[
            [
                [
                    151.29416017849223,
                    -33.740609827816364
                ],
                [
                    151.3173344643809,
                    -33.740609827816364
                ],
                [
                    151.3173344643809,
                    -33.69449020757253
                ],
                [
                    151.29416017849223,
                    -33.69449020757253
                ],
                [
                    151.29416017849223,
                    -33.740609827816364
                ]
            ]
        ]
    }
}

In [79]:
cloud_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        'lt': 0.1
    }
}

In [101]:
and_filter = {
    "type": 'AndFilter',
    "config": [geometry, data_filter, cloud_filter]
}

p(and_filter)

{
  "type": "AndFilter",
  "config": [
    {
      "type": "GeometryFilter",
      "field_name": "geometry",
      "config": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              151.29416017849223,
              -33.740609827816364
            ],
            [
              151.3173344643809,
              -33.740609827816364
            ],
            [
              151.3173344643809,
              -33.69449020757253
            ],
            [
              151.29416017849223,
              -33.69449020757253
            ],
            [
              151.29416017849223,
              -33.740609827816364
            ]
          ]
        ]
      }
    },
    {
      "type": "DateRangeFilter",
      "field_name": "acquired",
      "config": {
        "gte": "2017-11-30T00:00:00.000Z",
        "lte": "2018-01-30T00:00:00.000Z"
      }
    },
    {
      "type": "RangeFilter",
      "field_name": "cloud_cover",
      "config": {
        "lt": 0

# Get id of products we want

In [102]:
item_types = ['PSScene']
quicksearch_url = "{}/quick-search".format(URL)


request = {
    "item_types": item_types,
    "filter": and_filter
}

res = session.post(quicksearch_url, json=request)

geojson = res.json()

p(geojson)

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

# Find out which ones we have permissions for

In [70]:
feature = features[0]

print(feature['id'])
p(feature['_permissions'])


20190424_013129_50_106d
[
  "assets.basic_analytic_4b:download",
  "assets.basic_analytic_4b_rpc:download",
  "assets.basic_analytic_4b_xml:download",
  "assets.basic_udm2:download",
  "assets.ortho_analytic_4b:download",
  "assets.ortho_analytic_4b_sr:download",
  "assets.ortho_analytic_4b_xml:download",
  "assets.ortho_udm2:download",
  "assets.ortho_visual:download"
]


# Donwload with orders API

In [103]:
data_url =  'https://api.planet.com/data/v1'
data_url_quick_search = f'{data_url}/quick-search'
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
auth = requests.auth.HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(data_url, auth=auth)
response

<Response [200]>

In [104]:
# get the ids that we are interested in
item_type = "PSScene"

search_request = {
    "item_types": [item_type],
    "filter": and_filter
}

search_result = requests.post(
    data_url_quick_search,
    auth = requests.auth.HTTPBasicAuth(PLANET_API_KEY, ''),
    json=search_request
)

image_ids = [feature['id'] for feature in search_result.json()['features']]
print(image_ids)

['20180129_232041_0f51', '20180129_232040_0f51', '20180123_232028_1012', '20180121_232048_1008', '20180121_232047_1008', '20180120_011314_0f2e', '20180119_231915_1034', '20180118_231905_102d', '20180118_231907_102d', '20180118_232244_0e14', '20180118_232243_0e14', '20180118_011229_1054', '20180118_011230_1054', '20180113_232146_100e', '20180106_232029_102e', '20180106_232028_102e', '20180105_231913_103e', '20180105_231912_103e', '20180104_231910_100c', '20180104_231909_100c', '20171231_231932_1029', '20171231_231933_1029', '20171228_231952_1038', '20171228_231951_1038', '20171224_011751_103f', '20171223_232011_1015', '20171223_232010_1015', '20171223_231752_1039', '20171222_231829_101f', '20171222_231830_101f', '20171215_231806_0f4e', '20171215_231805_0f4e', '20171213_231843_100c', '20171213_231844_100c', '20171213_231741_0f18', '20171213_231742_0f18', '20171212_232027_0e26', '20171212_232026_0e26', '20171211_231800_1004', '20171211_231758_1004', '20171210_231903_100a', '20171210_23190

In [105]:
# authentificase orders the images for the ids

auth = requests.auth.HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
print(f"recent orders: {[order['name'] for order in response.json()['orders']]}")

recent orders: ['tiwaipoint', 'austrailianarrabeen', 'austrailianarrabeen', 'austrailianarrabeen']


In [106]:
sitename = 'australianarrabeen'

products = [
    {
        'item_ids': image_ids,
        'item_type': "PSScene",
        "product_bundle":"analytic_udm2"
    }
]

request = {
    "name": sitename,
    "products":products,
    "delivery": {"single_archive": True, "archive_type": 'zip'}
}

In [110]:
headers = {'content-type': 'application/json'}

def place_order(request, auth):
    response = requests.post(orders_url, data=json.dumps(request), auth=auth, headers=headers)

    if response.status_code in (200, 201):
        print("✅ Order placed successfully")
        order_id = response.json()['id']
        print(f"Order ID: {order_id}")
        order_url = orders_url + '/' + order_id
        return order_url
    else:
        print("❌ Failed to place order")
        print(f"Status code: {response.status_code}")
        try:
            print("Error details:", json.dumps(response.json(), indent=2))
        except Exception:
            print("Raw response:", response.text)
        return None

In [111]:
order_url = place_order(request, session.auth)

❌ Failed to place order
Status code: 400
Error details: {
  "field": {
    "Details": [
      {
        "message": "no access to assets: PSScene/20180118_231905_102d/[ortho_analytic_4b ortho_analytic_4b_xml]"
      }
    ]
  },
  "general": [
    {
      "message": "Unable to accept order"
    }
  ]
}


In [ ]:
def poll_for_success(order_url, auth, num_loops=30):
    count = 0
    while(count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        end_states = ['success', 'failed', 'partial']
        if state in end_states:
            break
        time.sleep(10)
        
poll_for_success(order_url, session.auth)

In [3]:
def write_api_key_file(api_key:str, overwrite:bool=False, data_dir:str=os.path.join(os.getcwd(), 'data')):
    sites_dir = os.path.join(data_dir, 'sites')
    if not os.path.exists(sites_dir): os.mkdir(sites_dir)
    
    file_path = os.path.join(sites_dir, 'PlanetScope_API_key.txt')
    if overwrite or not os.path.exists(file_path):
        # if we want to overwrite or if it doesnt exsist we will need to make it
        with open(file_path, "w") as file:
            file.write(api_key)